In [1]:
import os
import shutil
import sys
import glob
import pickle
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split

from pathlib import Path

In [2]:
SEED = 42
if SEED is not None:
    np.random.seed(SEED)
    random.seed(SEED)
    tf.random.set_random_seed(SEED)

In [3]:
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names

## Чтение данных

In [4]:
DATA_PATH = Path('data') / 'criteo-part'

TRAIN_PATH = str(DATA_PATH / 'train.csv')

In [5]:
train = pd.read_csv(TRAIN_PATH)

In [6]:
train.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,id
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [7]:
train = train.rename(columns=lambda x: x.replace('_', '').upper())

In [8]:
train.head()

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C31,C32,C33,C34,C35,C36,C37,C38,C39,ID
0,1,0.0,-1,NaN,NaN,1465.0,0.0,17.0,0.0,4.0,...,e5f8f18f,NaN,NaN,f3ddd519,NaN,32c7478e,b34f3128,NaN,NaN,12
1,1,0.0,1,20.0,16.0,1548.0,93.0,42.0,32.0,912.0,...,1f868fdd,21ddcdc9,a458ea53,7eee76d1,NaN,32c7478e,9af06ad9,9d93af03,cdfe5ab7,26
2,0,8.0,0,15.0,20.0,115.0,24.0,8.0,23.0,24.0,...,1304f63b,21ddcdc9,b1252a9d,07b2853e,NaN,32c7478e,94bde4f2,010f6491,09b76f8d,39
3,1,88.0,319,NaN,4.0,5.0,4.0,89.0,40.0,88.0,...,bbf70d82,NaN,NaN,16e2e3b3,NaN,32c7478e,d859b4dd,NaN,NaN,41
4,0,0.0,53,NaN,10.0,6550.0,98.0,34.0,11.0,349.0,...,fa0643ee,21ddcdc9,b1252a9d,0094bc78,NaN,32c7478e,29ece3ed,001f3601,402185f3,85


In [9]:
num_columns = ['C{}'.format(i) for i in range(1, 14)]
cat_columns = ['C{}'.format(i) for i in range(14, 40)]
target = ['C0']
len(num_columns), len(cat_columns)

(13, 26)

In [10]:
X, y = train.drop(columns=['C0', 'ID']), train['C0']

# Data preprocessing

In [11]:
X[num_columns] = X[num_columns].fillna(0)
X[cat_columns] = X[cat_columns].fillna('-1')

In [12]:
from sklearn.preprocessing import MinMaxScaler
import category_encoders as ce

In [13]:
X_encoded = X.copy()

In [14]:
mms = MinMaxScaler(feature_range=(0, 1))

In [15]:
X_encoded[num_columns] = mms.fit_transform(X_encoded[num_columns])

In [16]:
encoder = ce.OrdinalEncoder(cols=cat_columns)

In [17]:
X_encoded = encoder.fit_transform(X_encoded)

C:\Anaconda3\envs\tf_fibi\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [18]:
X_train_encoded, X_valid_encoded, y_train, y_valid = train_test_split(X_encoded, y, stratify=y, random_state=SEED)

In [19]:
X_train_encoded.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39
597429,0.000000,0.033123,0.000427,0.000000,5.553482e-04,0.000669,0.000202,0.004752,0.001966,0.000000,...,5,213,2,4,7150,1,1,229,20,166
1768963,0.000000,0.000136,0.000412,0.021665,5.553482e-04,0.002609,0.000101,0.005741,0.004450,0.000000,...,9,152,20,2,94823,1,4,56,12,44
1913948,0.000000,0.000861,0.000137,0.007982,8.473158e-04,0.000129,0.003737,0.001386,0.002949,0.000000,...,1,232,15,3,14484,1,3,261,3,191
266546,0.000000,0.000181,0.000076,0.007982,2.668111e-06,0.000000,0.000000,0.001782,0.000362,0.000000,...,7,118,1,1,275,4,2,112,1,1
800178,0.000519,0.000181,0.000031,0.000000,7.623174e-07,0.000000,0.000303,0.000198,0.000052,0.222222,...,1,33,36,3,226291,1,1,4950,4,3822


In [20]:
sparse_features = [SparseFeat(feat, X_encoded[feat].nunique() + 1) for feat in cat_columns]
# or using hashing
# sparse_features = [SparseFeat(feat, vocabulary_size=1000, embedding_dim=4, use_hash=True, dtype='string')  for feat in cat_columns]
dense_features = [DenseFeat(feat, 1) for feat in num_columns]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
fixlen_feature_columns = sparse_features + dense_features

In [22]:
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [23]:
train_model_input = {name:X_train_encoded[name] for name in feature_names}

In [24]:
test_model_input = {name:X_valid_encoded[name] for name in feature_names}

# Model training

In [25]:
from deepctr.models import DeepFM, CCPM, PNN, WDL, MLR, NFM, AFM, DCN, xDeepFM, AutoInt, ONN, FiBiNET
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import roc_auc_score, log_loss

In [26]:
def get_metrics(groups) -> pd.DataFrame:
    metrics =  pd.DataFrame.from_records(groups, columns=groups.keys(), index=list(all_metrics.values())[0].keys())
    metrics.index.name = 'metric'
    return metrics

def get_ate(groups, control_name) -> pd.DataFrame:
    """Get Average Treatment Effect
    groups - dictionary where keys - names of models, values - dicts of pairs <metric_name>, <metric_value>
    control_name - name of baseline model
    
    return pd.DataFrame (rows corresponds to metrics, cols corresponds to models and ATE with respect to control)
    """
    metrics = get_metrics(groups)
    return metrics.subtract(metrics[control_name], axis='index').drop(columns=control_name) * 100


all_metrics = {}

In [27]:
def test_model(model, train_model_input, test_model_input, y_train, y_valid, batch_size=4096 * 2, model_name='model'):
    checkpoint_path = f'models/criteo/tensorflow/sparse/{model_name}'
    os.makedirs(checkpoint_path, exist_ok=True)
    checkpoint_path += '/model.ckpt'
    
    model.compile("adam", "binary_crossentropy",
              metrics=["binary_crossentropy", tf.keras.metrics.AUC()])
    
    es = EarlyStopping(monitor='val_auc', verbose=1, patience=2, mode='max')
    mdckpt = ModelCheckpoint(filepath=checkpoint_path, monitor='val_auc', mode='max', save_best_only=True, save_weights_only=True)
    
    history = model.fit(train_model_input, y_train,
                        batch_size=batch_size, epochs=15, verbose=2, validation_split=0.2, callbacks=[es, mdckpt])
    
    model.load_weights(checkpoint_path)
    
    y_valid_predicted = model.predict(test_model_input, batch_size=batch_size).squeeze()
    
    scores = {
        'roc-auc': roc_auc_score(y_valid, y_valid_predicted),
        'log-loss': log_loss(y_valid, y_valid_predicted)        
    }
    del model
    tf.keras.backend.clear_session()
    return scores

## CCPM (Convolutional Click Prediction Model)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/CCPM.png" alt="drawing" width="900"/>

In [28]:
ccpm_model = CCPM(linear_feature_columns, sparse_features, dnn_dropout=0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [29]:
all_metrics['CCPM'] = test_model(ccpm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='CCPM')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 101s - loss: 0.5191 - binary_crossentropy: 0.5159 - auc: 0.7063 - val_loss: 0.4942 - val_binary_crossentropy: 0.4864 - val_auc: 0.7560
Epoch 2/15
2198958/2198958 - 98s - loss: 0.4838 - binary_crossentropy: 0.4709 - auc: 0.7761 - val_loss: 0.5004 - val_binary_crossentropy: 0.4838 - val_auc: 0.7595
Epoch 3/15
2198958/2198958 - 98s - loss: 0.4565 - binary_crossentropy: 0.4339 - auc: 0.8179 - val_loss: 0.5283 - val_binary_crossentropy: 0.5000 - val_auc: 0.7466
Epoch 4/15
2198958/2198958 - 98s - loss: 0.3953 - binary_crossentropy: 0.3669 - auc: 0.8731 - val_loss: 0.5610 - val_binary_crossentropy: 0.5309 - val_auc: 0.7346
Epoch 00004: early stopping


## PNN (Product-based Neural Network)
<!-- ![pic](https://deepctr-torch.readthedocs.io/en/latest/_images/PNN.png) -->
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/PNN.png" alt="drawing" width="900"/>

In [30]:
pnn_model = PNN(dnn_feature_columns, dnn_dropout=0.5)

In [31]:
all_metrics['PNN'] = test_model(pnn_model, train_model_input, test_model_input, y_train.values, y_valid,  model_name='PNN')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 14s - loss: 0.5097 - binary_crossentropy: 0.5061 - auc: 0.7236 - val_loss: 0.4828 - val_binary_crossentropy: 0.4756 - val_auc: 0.7711
Epoch 2/15
2198958/2198958 - 12s - loss: 0.4573 - binary_crossentropy: 0.4422 - auc: 0.8110 - val_loss: 0.5067 - val_binary_crossentropy: 0.4840 - val_auc: 0.7633
Epoch 3/15
2198958/2198958 - 12s - loss: 0.3688 - binary_crossentropy: 0.3473 - auc: 0.8870 - val_loss: 0.5413 - val_binary_crossentropy: 0.5202 - val_auc: 0.7526
Epoch 00003: early stopping


## Wide & Deep
<!-- ![pic](https://deepctr-torch.readthedocs.io/en/latest/_images/WDL.png) -->
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/WDL.png" alt="drawing" width="900"/>

In [32]:
wdl_model = WDL(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [33]:
all_metrics['WDL'] = test_model(wdl_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='WDL')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 13s - loss: 0.5047 - binary_crossentropy: 0.5007 - auc: 0.7322 - val_loss: 0.4831 - val_binary_crossentropy: 0.4750 - val_auc: 0.7719
Epoch 2/15
2198958/2198958 - 11s - loss: 0.4595 - binary_crossentropy: 0.4432 - auc: 0.8103 - val_loss: 0.5049 - val_binary_crossentropy: 0.4807 - val_auc: 0.7659
Epoch 3/15
2198958/2198958 - 11s - loss: 0.3712 - binary_crossentropy: 0.3485 - auc: 0.8859 - val_loss: 0.5446 - val_binary_crossentropy: 0.5225 - val_auc: 0.7487
Epoch 00003: early stopping


## DeepFM
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/DeepFM.png" alt="drawing" width="900"/>

In [34]:
deep_fm_model = DeepFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [35]:
all_metrics['DeepFM'] = test_model(deep_fm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='DeepFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 13s - loss: 0.5048 - binary_crossentropy: 0.5007 - auc: 0.7320 - val_loss: 0.4832 - val_binary_crossentropy: 0.4750 - val_auc: 0.7719
Epoch 2/15
2198958/2198958 - 12s - loss: 0.4616 - binary_crossentropy: 0.4453 - auc: 0.8079 - val_loss: 0.5043 - val_binary_crossentropy: 0.4803 - val_auc: 0.7663
Epoch 3/15
2198958/2198958 - 11s - loss: 0.3726 - binary_crossentropy: 0.3498 - auc: 0.8855 - val_loss: 0.5426 - val_binary_crossentropy: 0.5201 - val_auc: 0.7508
Epoch 00003: early stopping


## MLR(Mixed Logistic Regression/Piece-wise Linear Model)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/MLR.png" alt="drawing" width="900"/>

In [36]:
mlr_model = MLR(linear_feature_columns, dnn_feature_columns)

In [37]:
all_metrics['MLR'] = test_model(mlr_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='MLR')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 20s - loss: 0.5362 - binary_crossentropy: 0.5337 - auc: 0.6770 - val_loss: 0.5053 - val_binary_crossentropy: 0.5013 - val_auc: 0.7408
Epoch 2/15
2198958/2198958 - 16s - loss: 0.4956 - binary_crossentropy: 0.4903 - auc: 0.7550 - val_loss: 0.4976 - val_binary_crossentropy: 0.4915 - val_auc: 0.7510
Epoch 3/15
2198958/2198958 - 15s - loss: 0.4894 - binary_crossentropy: 0.4823 - auc: 0.7646 - val_loss: 0.4953 - val_binary_crossentropy: 0.4879 - val_auc: 0.7552
Epoch 4/15
2198958/2198958 - 16s - loss: 0.4867 - binary_crossentropy: 0.4786 - auc: 0.7690 - val_loss: 0.4941 - val_binary_crossentropy: 0.4859 - val_auc: 0.7578
Epoch 5/15
2198958/2198958 - 16s - loss: 0.4851 - binary_crossentropy: 0.4764 - auc: 0.7718 - val_loss: 0.4934 - val_binary_crossentropy: 0.4846 - val_auc: 0.7596
Epoch 6/15
2198958/2198958 - 16s - loss: 0.4841 - binary_crossentropy: 0.4748 - auc: 0.7738 - val_loss: 0.4928 - val_binary_crossent

## NFM (Neural Factorization Machine)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/NFM.png" alt="drawing" width="900"/>

In [38]:
nfm_model = NFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [39]:
all_metrics['NFM'] = test_model(nfm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='NFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 13s - loss: 0.5078 - binary_crossentropy: 0.5037 - auc: 0.7276 - val_loss: 0.4847 - val_binary_crossentropy: 0.4760 - val_auc: 0.7710
Epoch 2/15
2198958/2198958 - 12s - loss: 0.4703 - binary_crossentropy: 0.4567 - auc: 0.7946 - val_loss: 0.4912 - val_binary_crossentropy: 0.4739 - val_auc: 0.7738
Epoch 3/15
2198958/2198958 - 12s - loss: 0.4389 - binary_crossentropy: 0.4167 - auc: 0.8351 - val_loss: 0.5224 - val_binary_crossentropy: 0.4949 - val_auc: 0.7595
Epoch 4/15
2198958/2198958 - 12s - loss: 0.3886 - binary_crossentropy: 0.3610 - auc: 0.8779 - val_loss: 0.5678 - val_binary_crossentropy: 0.5386 - val_auc: 0.7446
Epoch 00004: early stopping


## AFM (Attentional Factorization Machine)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/AFM.png" alt="drawing" width="900"/>

In [40]:
afm_model = AFM(linear_feature_columns, sparse_features, afm_dropout=0.5)

Instructions for updating:
dim is deprecated, use axis instead


In [41]:
all_metrics['AFM'] = test_model(afm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='AFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 20s - loss: 0.5428 - binary_crossentropy: 0.5414 - auc: 0.6623 - val_loss: 0.5132 - val_binary_crossentropy: 0.5102 - val_auc: 0.7290
Epoch 2/15
2198958/2198958 - 19s - loss: 0.5016 - binary_crossentropy: 0.4971 - auc: 0.7470 - val_loss: 0.5019 - val_binary_crossentropy: 0.4964 - val_auc: 0.7441
Epoch 3/15
2198958/2198958 - 18s - loss: 0.4932 - binary_crossentropy: 0.4867 - auc: 0.7584 - val_loss: 0.4985 - val_binary_crossentropy: 0.4915 - val_auc: 0.7501
Epoch 4/15
2198958/2198958 - 19s - loss: 0.4899 - binary_crossentropy: 0.4822 - auc: 0.7643 - val_loss: 0.4969 - val_binary_crossentropy: 0.4890 - val_auc: 0.7533
Epoch 5/15
2198958/2198958 - 18s - loss: 0.4879 - binary_crossentropy: 0.4794 - auc: 0.7678 - val_loss: 0.4958 - val_binary_crossentropy: 0.4873 - val_auc: 0.7556
Epoch 6/15
2198958/2198958 - 19s - loss: 0.4865 - binary_crossentropy: 0.4775 - auc: 0.7702 - val_loss: 0.4950 - val_binary_crossent

## DCN (Deep & Cross Network)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/DCN.png" alt="drawing" width="900"/>

In [42]:
dcn_model = DCN(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [43]:
all_metrics['DCN'] = test_model(dcn_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='DCN')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 15s - loss: 0.5033 - binary_crossentropy: 0.4994 - auc: 0.7342 - val_loss: 0.4832 - val_binary_crossentropy: 0.4752 - val_auc: 0.7718
Epoch 2/15
2198958/2198958 - 13s - loss: 0.4628 - binary_crossentropy: 0.4471 - auc: 0.8064 - val_loss: 0.5025 - val_binary_crossentropy: 0.4798 - val_auc: 0.7673
Epoch 3/15
2198958/2198958 - 13s - loss: 0.3696 - binary_crossentropy: 0.3481 - auc: 0.8865 - val_loss: 0.5356 - val_binary_crossentropy: 0.5142 - val_auc: 0.7532
Epoch 00003: early stopping


## xDeepFM
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/xDeepFM.png" alt="drawing" width="900"/>

In [44]:
xdeep_fm_model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)

In [45]:
all_metrics['xDeepFM'] = test_model(xdeep_fm_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='xDeepFM')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 25s - loss: 0.4983 - binary_crossentropy: 0.4944 - auc: 0.7417 - val_loss: 0.4805 - val_binary_crossentropy: 0.4725 - val_auc: 0.7750
Epoch 2/15
2198958/2198958 - 22s - loss: 0.4331 - binary_crossentropy: 0.4171 - auc: 0.8351 - val_loss: 0.5180 - val_binary_crossentropy: 0.4942 - val_auc: 0.7611
Epoch 3/15
2198958/2198958 - 22s - loss: 0.3598 - binary_crossentropy: 0.3381 - auc: 0.8916 - val_loss: 0.5429 - val_binary_crossentropy: 0.5225 - val_auc: 0.7507
Epoch 00003: early stopping


## AutoInt(Automatic Feature Interaction)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/AutoInt.png" alt="drawing" width="900"/>

In [46]:
auto_int_model = AutoInt(linear_feature_columns, dnn_feature_columns, dnn_dropout=0.5)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [47]:
all_metrics['AutoInt'] = test_model(auto_int_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='AutoInt')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 29s - loss: 0.5011 - binary_crossentropy: 0.4972 - auc: 0.7375 - val_loss: 0.4815 - val_binary_crossentropy: 0.4735 - val_auc: 0.7737
Epoch 2/15
2198958/2198958 - 27s - loss: 0.4414 - binary_crossentropy: 0.4251 - auc: 0.8280 - val_loss: 0.5189 - val_binary_crossentropy: 0.4940 - val_auc: 0.7582
Epoch 3/15
2198958/2198958 - 27s - loss: 0.3665 - binary_crossentropy: 0.3434 - auc: 0.8880 - val_loss: 0.5654 - val_binary_crossentropy: 0.5436 - val_auc: 0.7367
Epoch 00003: early stopping


## FiBiNET(Feature Importance and Bilinear feature Interaction NETwork)
<img src="https://deepctr-torch.readthedocs.io/en/latest/_images/FiBiNET.png" alt="drawing" width="900"/>

In [48]:
fibinet_interaction_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='interaction', dnn_dropout=0.5)

In [49]:
all_metrics['FiBiNET-inter'] = test_model(fibinet_interaction_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-inter')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 49s - loss: 0.5079 - binary_crossentropy: 0.5045 - auc: 0.7257 - val_loss: 0.4805 - val_binary_crossentropy: 0.4734 - val_auc: 0.7746
Epoch 2/15
2198958/2198958 - 32s - loss: 0.4587 - binary_crossentropy: 0.4449 - auc: 0.8084 - val_loss: 0.5014 - val_binary_crossentropy: 0.4813 - val_auc: 0.7690
Epoch 3/15
2198958/2198958 - 32s - loss: 0.3650 - binary_crossentropy: 0.3453 - auc: 0.8885 - val_loss: 0.5386 - val_binary_crossentropy: 0.5179 - val_auc: 0.7537
Epoch 00003: early stopping


In [50]:
fibinet_all_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='all', dnn_dropout=0.5)

In [51]:
all_metrics['FiBiNET-all'] = test_model(fibinet_all_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-all')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 26s - loss: 0.5080 - binary_crossentropy: 0.5047 - auc: 0.7253 - val_loss: 0.4811 - val_binary_crossentropy: 0.4742 - val_auc: 0.7732
Epoch 2/15
2198958/2198958 - 20s - loss: 0.4609 - binary_crossentropy: 0.4471 - auc: 0.8059 - val_loss: 0.4974 - val_binary_crossentropy: 0.4771 - val_auc: 0.7712
Epoch 3/15
2198958/2198958 - 20s - loss: 0.3685 - binary_crossentropy: 0.3480 - auc: 0.8870 - val_loss: 0.5328 - val_binary_crossentropy: 0.5112 - val_auc: 0.7560
Epoch 00003: early stopping


In [52]:
fibinet_each_model = FiBiNET(linear_feature_columns, dnn_feature_columns, bilinear_type='each', dnn_dropout=0.5)

In [53]:
all_metrics['FiBiNET-each'] = test_model(fibinet_each_model, train_model_input, test_model_input, y_train.values, y_valid, model_name='FiBiNet-each')

Train on 2198958 samples, validate on 549740 samples
Epoch 1/15
2198958/2198958 - 26s - loss: 0.5080 - binary_crossentropy: 0.5046 - auc: 0.7253 - val_loss: 0.4806 - val_binary_crossentropy: 0.4736 - val_auc: 0.7747
Epoch 2/15
2198958/2198958 - 20s - loss: 0.4632 - binary_crossentropy: 0.4495 - auc: 0.8033 - val_loss: 0.4950 - val_binary_crossentropy: 0.4754 - val_auc: 0.7731
Epoch 3/15
2198958/2198958 - 20s - loss: 0.3705 - binary_crossentropy: 0.3509 - auc: 0.8853 - val_loss: 0.5272 - val_binary_crossentropy: 0.5066 - val_auc: 0.7581
Epoch 00003: early stopping


# Результаты

In [54]:
get_metrics(all_metrics)

,CCPM,PNN,WDL,DeepFM,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-inter,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,,
roc-auc,0.760732,0.772439,0.773036,0.773102,0.767278,0.774848,0.766106,0.772785,0.776003,0.774557,0.775675,0.774398,0.775636
log-loss,0.482173,0.473998,0.473490,0.473316,0.478206,0.472445,0.478988,0.473727,0.471138,0.472109,0.471906,0.472636,0.472175


In [55]:
get_ate(all_metrics, control_name='DeepFM')

,CCPM,PNN,WDL,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-inter,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,
roc-auc,-1.237039,-0.066382,-0.006662,-0.582453,0.174585,-0.699617,-0.031779,0.290032,0.14542,0.257312,0.129543,0.253338
log-loss,0.885681,0.068183,0.017370,0.488952,-0.087124,0.567238,0.041056,-0.217757,-0.12069,-0.141011,-0.068035,-0.114085


In [56]:
get_ate(all_metrics, control_name='FiBiNET-inter')

,CCPM,PNN,WDL,DeepFM,MLR,NFM,AFM,DCN,xDeepFM,AutoInt,FiBiNET-all,FiBiNET-each
metric,,,,,,,,,,,,
roc-auc,-1.494351,-0.323694,-0.263974,-0.257312,-0.839765,-0.082727,-0.956929,-0.289091,0.032720,-0.111892,-0.127769,-0.003974
log-loss,1.026692,0.209194,0.158381,0.141011,0.629964,0.053887,0.708250,0.182068,-0.076746,0.020321,0.072977,0.026927
